************************************************************
# Final

In [1]:
import pandas as pd
import numpy as np


from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import re

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
article_path = "/content/drive/MyDrive/INTER_IIT_9.0_Bridgei2i/Development Data/dev_data_article.xlsx"
tweet_path = "/content/drive/MyDrive/INTER_IIT_9.0_Bridgei2i/Development Data/dev_data_tweet.xlsx"

PreProcessing


In [4]:

df_tweet = pd.read_excel(tweet_path)
df_tweet=df_tweet.dropna()
df_tweet.columns = ['Id', 'text', 'class']
df_tweet.drop_duplicates(subset=['text'], inplace=True)
df_tweet[1570:1590]

,Id,text,class
2523,tweet_2524,"SBI दे रहा सबसे सस्ता Home Loan, 31 मार्च तक न...",0
2524,tweet_2525,RT @AshishK51455345: हर बड़े मुद्दे पर चीन को ...,0
2525,tweet_2526,"Santa ne bank loan pe 1 car li, Loan nahi chuk...",0
2526,tweet_2527,QT @Chamanlal_: सेंट्रल बैंक कश्मीरी गेट के ब्...,0
2530,tweet_2531,@srivatsayb Ghar banane ke liye tu ek bank se ...,0
2531,tweet_2532,RT @anupam0044: @AdvocateBLJain @capankajssm भ...,0
2532,tweet_2533,RT @birendra112016: @AdvocateBLJain @capankajs...,0
2533,tweet_2534,RT @ArunChe12360653: @SonuSood hi sir Mera naa...,0
2534,tweet_2535,@SonuSood Sir Aapse bahut help manga Apne medi...,0
2537,tweet_2538,Sir meine loan mein 1 lakh liya tha ab operati...,0


In [5]:
df_article = pd.read_excel(article_path)
df_article.columns = ['Id', 'text','headline', 'class']
df_article = df_article.dropna()
df_article.drop_duplicates(subset=['text'] , inplace = True)
df_article.tail()


,Id,text,headline,class
3995,article_3996,All Hindi Bloggers List with All Detail\nMay 1...,All Hindi Bloggers List with All Detail,0
3996,article_3997,"Amazon Flex se Part Time me Earn kare ₹15,000 ...","Amazon Flex se Part Time me Earn kare ₹15,000 ...",0
3997,article_3998,Bad Link Building Strategies se Kaise Bachaye ...,Bad Link Building Strategies se Kaise Bachaye ...,0
3998,article_3999,Badiya Comment karte Samay Rakhe in Baat ka Dh...,Badiya Comment karte Samay Rakhe in Baat ka Dh...,0
3999,article_4000,Best Blogger Blog Theme का चुनाव कैसे करे ? [ ...,Best Blogger Blog Theme का चुनाव कैसे करे ? [ ...,0


In [6]:

def clean_text(text):
  ans = []
  for tweet in text:
    tweet = tweet.lower()
    tweet = re.sub(r'[^\w\s]', '', tweet) 
    tweet = re.sub(r'^rt[\s]+','',tweet)
    tweet = re.sub(r'^qt[\s]+','',tweet)
    tweet = re.sub(r'https?:\/\/.*[\s]*','',tweet)
    tweet = re.sub(r'(#|@|:|>)',' ',tweet)
    tweet = re.sub(r'[\s]+',' ',tweet)
    # tweet = re.sub(r'[^\x00-\x7F]+|\x0c',' ', tweet) # remove all non-XML-compatible character ie the emojis
    ans.append(tweet)
  return ans


cleaned_text = clean_text(df_tweet['text'])
df_tweet['cleaned_text'] = cleaned_text

display(df_tweet)


cleaned_text = clean_text(df_article['text'])
df_article['cleaned_text'] = cleaned_text

display(df_article.head())




,Id,text,class,cleaned_text
0,tweet_0001,You'll 💜 my #PitchWars book if you like: 🦋 hat...,0,youll my pitchwars book if you like hatetolove...
1,tweet_0002,RT @SkySportsNews: 🚨 Breaking: #WBA have reach...,0,skysportsnews breaking wba have reached a verb...
2,tweet_0003,RT @stealyoman_cuso: really says a lot about s...,0,stealyoman_cuso really says a lot about societ...
3,tweet_0004,RT @PGtzsche1: HPV vaccines increased serious ...,0,pgtzsche1 hpv vaccines increased serious nervo...
4,tweet_0005,Ramaphosa says if you are positive you must se...,0,ramaphosa says if you are positive you must se...
...,...,...,...,...
3991,tweet_3992,#Realme ने 5G सेगमेंट में एक धांसू फोन #Realme...,1,realme न 5g सगमट म एक धस फन realmev15 लनच कय ह...
3992,tweet_3993,"भारत में लॉन्च हुआ Xiaomi Mi 10i, यह है 2021 क...",1,भरत म लनच हआ xiaomi mi 10i यह ह 2021 क सबस ससत...
3997,tweet_3998,चीनी मोबाइल कंपनी रियलमी ने अपना Realme V15 5G...,1,चन मबइल कपन रयलम न अपन realme v15 5g समरटफन लन...
3998,tweet_3999,QT @AnjaliSingh_IN: Aapne phone hi galat choos...,1,anjalisingh_in aapne phone hi galat choose kr ...


,Id,text,headline,class,cleaned_text
0,article_0001,Digitisation is one of the key buzzwords in th...,Pakistan’s digital landscape post-Covid,0,digitisation is one of the key buzzwords in th...
1,article_0002,Increase in tolerance limit up to 120 per cent...,Affordable housing gets shot in the arm,0,increase in tolerance limit up to 120 per cent...
2,article_0003,Home > News > World Sports News\n\nJonas Lossl...,Jonas Lossl leaves Everton to rejoin first clu...,0,home news world sports news jonas lossl leaves...
3,article_0004,"Source: Agfax.com\n\nBy Keith Brown, DTN Contr...",DTN Cotton Closing: Cotton Higher on Commoditi...,0,source agfaxcom by keith brown dtn contributin...
4,article_0005,"United Nations, Feb 4: The Serum Institute of ...","Serum Institute of India, UNICEF enter into lo...",0,united nations feb 4 the serum institute of in...


In [7]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.hi import STOP_WORDS as STOP_WORDS_HI

nlp = spacy.load("en_core_web_sm")

In [8]:

def preprocess_text(tweet):
  doc = nlp(tweet)
  token_list_for_tweet = [token for token in doc]
  hindi_removed = [token for token in doc if token.text not in STOP_WORDS_HI]
  filtered_tokens = [token for token in hindi_removed if not token.is_stop]
  lemmatized_tweet_list=[token.lemma_ for token in filtered_tokens if token.is_punct==False]
  return lemmatized_tweet_list

bag_of_words = []
for text in df_tweet['cleaned_text']:

  x = preprocess_text(text)
  bag_of_words.append(str(x))


df_tweet['bag_of_words'] = bag_of_words

In [9]:
df_tweet

,Id,text,class,cleaned_text,bag_of_words
0,tweet_0001,You'll 💜 my #PitchWars book if you like: 🦋 hat...,0,youll my pitchwars book if you like hatetolove...,"['will', 'pitchwar', 'book', 'like', 'hatetolo..."
1,tweet_0002,RT @SkySportsNews: 🚨 Breaking: #WBA have reach...,0,skysportsnews breaking wba have reached a verb...,"['skysportsnews', 'break', 'wba', 'reach', 've..."
2,tweet_0003,RT @stealyoman_cuso: really says a lot about s...,0,stealyoman_cuso really says a lot about societ...,"['stealyoman_cuso', 'say', 'lot', 'society', '..."
3,tweet_0004,RT @PGtzsche1: HPV vaccines increased serious ...,0,pgtzsche1 hpv vaccines increased serious nervo...,"['pgtzsche1', 'hpv', 'vaccine', 'increase', 'n..."
4,tweet_0005,Ramaphosa says if you are positive you must se...,0,ramaphosa says if you are positive you must se...,"['ramaphosa', 'say', 'positive', 'self', 'isol..."
...,...,...,...,...,...
3991,tweet_3992,#Realme ने 5G सेगमेंट में एक धांसू फोन #Realme...,1,realme न 5g सगमट म एक धस फन realmev15 लनच कय ह...,"['realme', '5', 'g', 'सगमट', 'म', 'धस', 'फन', ..."
3992,tweet_3993,"भारत में लॉन्च हुआ Xiaomi Mi 10i, यह है 2021 क...",1,भरत म लनच हआ xiaomi mi 10i यह ह 2021 क सबस ससत...,"['भरत', 'म', 'लनच', 'हआ', 'xiaomi', 'mi', '10i..."
3997,tweet_3998,चीनी मोबाइल कंपनी रियलमी ने अपना Realme V15 5G...,1,चन मबइल कपन रयलम न अपन realme v15 5g समरटफन लन...,"['चन', 'मबइल', 'कपन', 'रयलम', 'अपन', 'realme',..."
3998,tweet_3999,QT @AnjaliSingh_IN: Aapne phone hi galat choos...,1,anjalisingh_in aapne phone hi galat choose kr ...,"['anjalisingh_in', 'aapne', 'phone', 'hi', 'ga..."


In [10]:
bag_of_words = []
for text in df_article['cleaned_text']:

  x = preprocess_text(text)
  bag_of_words.append(str(x))


df_article['bag_of_words'] = bag_of_words

In [11]:
df_article

,Id,text,headline,class,cleaned_text,bag_of_words
0,article_0001,Digitisation is one of the key buzzwords in th...,Pakistan’s digital landscape post-Covid,0,digitisation is one of the key buzzwords in th...,"['digitisation', 'key', 'buzzword', 'postcovid..."
1,article_0002,Increase in tolerance limit up to 120 per cent...,Affordable housing gets shot in the arm,0,increase in tolerance limit up to 120 per cent...,"['increase', 'tolerance', 'limit', '120', 'cen..."
2,article_0003,Home > News > World Sports News\n\nJonas Lossl...,Jonas Lossl leaves Everton to rejoin first clu...,0,home news world sports news jonas lossl leaves...,"['home', 'news', 'world', 'sports', 'news', 'j..."
3,article_0004,"Source: Agfax.com\n\nBy Keith Brown, DTN Contr...",DTN Cotton Closing: Cotton Higher on Commoditi...,0,source agfaxcom by keith brown dtn contributin...,"['source', 'agfaxcom', 'keith', 'brown', 'dtn'..."
4,article_0005,"United Nations, Feb 4: The Serum Institute of ...","Serum Institute of India, UNICEF enter into lo...",0,united nations feb 4 the serum institute of in...,"['united', 'nations', 'feb', '4', 'serum', 'in..."
...,...,...,...,...,...,...
3995,article_3996,All Hindi Bloggers List with All Detail\nMay 1...,All Hindi Bloggers List with All Detail,0,all hindi bloggers list with all detail may 15...,"['hindi', 'blogger', 'list', 'detail', '15', '..."
3996,article_3997,"Amazon Flex se Part Time me Earn kare ₹15,000 ...","Amazon Flex se Part Time me Earn kare ₹15,000 ...",0,amazon flex se part time me earn kare 15000 ea...,"['amazon', 'flex', 'se', 'time', 'earn', 'kare..."
3997,article_3998,Bad Link Building Strategies se Kaise Bachaye ...,Bad Link Building Strategies se Kaise Bachaye ...,0,bad link building strategies se kaise bachaye ...,"['bad', 'link', 'building', 'strategy', 'se', ..."
3998,article_3999,Badiya Comment karte Samay Rakhe in Baat ka Dh...,Badiya Comment karte Samay Rakhe in Baat ka Dh...,0,badiya comment karte samay rakhe in baat ka dh...,"['badiya', 'comment', 'karte', 'samay', 'rakhe..."


THEME CLASSIFIER

In [12]:
# train_X , test_X ,train_Y , test_Y  = model_selection.train_test_split(df_tweet['bag_of_words'],df_tweet['class'],test_size=0.3)

a = df_tweet["bag_of_words"]
b = df_article["bag_of_words"]

print(a.shape)
print(b.shape)

df = pd.DataFrame(a.append(b, ignore_index = True))

display(df)

c = df_tweet["class"]
d = df_article["class"]
df_class = pd.DataFrame(c.append(d, ignore_index = True))

display(df_class)



(2400,)
(3567,)


,bag_of_words
0,"['will', 'pitchwar', 'book', 'like', 'hatetolo..."
1,"['skysportsnews', 'break', 'wba', 'reach', 've..."
2,"['stealyoman_cuso', 'say', 'lot', 'society', '..."
3,"['pgtzsche1', 'hpv', 'vaccine', 'increase', 'n..."
4,"['ramaphosa', 'say', 'positive', 'self', 'isol..."
...,...
5962,"['hindi', 'blogger', 'list', 'detail', '15', '..."
5963,"['amazon', 'flex', 'se', 'time', 'earn', 'kare..."
5964,"['bad', 'link', 'building', 'strategy', 'se', ..."
5965,"['badiya', 'comment', 'karte', 'samay', 'rakhe..."


,class
0,0
1,0
2,0
3,0
4,0
...,...
5962,0
5963,0
5964,0
5965,0


In [13]:
Tfidf_vect = TfidfVectorizer()
Tfidf_vect.fit(df['bag_of_words'])
Train_X_Tfidf = Tfidf_vect.transform(df['bag_of_words'])

Train_X_Tfidf


# Test_X_Tfidf = Tfidf_vect.transform(test_X)

<5967x71230 sparse matrix of type '<class 'numpy.float64'>'
	with 713737 stored elements in Compressed Sparse Row format>

In [14]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,df_class["class"])


SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto', kernel='linear',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)

Testing on Testdataset


In [17]:
test_data_path = "/content/drive/MyDrive/INTER_IIT_9.0_Bridgei2i/LatestTestDataset/evaluation_data.xlsx"

In [18]:
df_test = pd.read_excel(test_data_path)
df_test

,Text_ID,Text
0,article_4001,Market Highlights\n\nAn automotive constant ve...
1,article_4002,Overview:\n\nBe it as a means for food or as p...
2,article_4003,"GenScript USA Inc. (U.S.), Horizon Discovery G..."
3,article_4004,New Delhi: Prime Minister Narendra Modi spoke ...
4,article_4005,"New Delhi, Feb 1 (PTI) The Delhi High Court Mo..."
...,...,...
427,tweet_4206,#ios 6.1 #for #iphone #very #bad and #iPhone5 ...
428,tweet_4207,One #bad thing of #XiaomiRedmiNote4 is the au...
429,tweet_4208,I have a concern over the service provided . ...
430,tweet_4209,#Bought Nokia6.1Plus in Oct2018 and having wor...


In [19]:
cleaned_text = clean_text(df_test['Text'])
df_test['cleaned_text'] = cleaned_text

In [20]:
df_test

,Text_ID,Text,cleaned_text
0,article_4001,Market Highlights\n\nAn automotive constant ve...,market highlights an automotive constant veloc...
1,article_4002,Overview:\n\nBe it as a means for food or as p...,overview be it as a means for food or as pets ...
2,article_4003,"GenScript USA Inc. (U.S.), Horizon Discovery G...",genscript usa inc us horizon discovery group p...
3,article_4004,New Delhi: Prime Minister Narendra Modi spoke ...,new delhi prime minister narendra modi spoke t...
4,article_4005,"New Delhi, Feb 1 (PTI) The Delhi High Court Mo...",new delhi feb 1 pti the delhi high court monda...
...,...,...,...
427,tweet_4206,#ios 6.1 #for #iphone #very #bad and #iPhone5 ...,ios 61 for iphone very bad and iphone5 not goo...
428,tweet_4207,One #bad thing of #XiaomiRedmiNote4 is the au...,one bad thing of xiaomiredminote4 is the auto ...
429,tweet_4208,I have a concern over the service provided . ...,i have a concern over the service provided pu...
430,tweet_4209,#Bought Nokia6.1Plus in Oct2018 and having wor...,bought nokia61plus in oct2018 and having worst...


In [21]:
bag_of_words = []
for text in df_test['cleaned_text']:

  x = preprocess_text(text)
  bag_of_words.append(str(x))


df_test['bag_of_words'] = bag_of_words

In [22]:
df_test["bag_of_words"]

0      ['market', 'highlight', 'automotive', 'constan...
1      ['overview', 'means', 'food', 'pet', 'animal',...
2      ['genscript', 'usa', 'inc', 'horizon', 'discov...
3      ['new', 'delhi', 'prime', 'minister', 'narendr...
4      ['new', 'delhi', 'feb', '1', 'pti', 'delhi', '...
                             ...                        
427    ['ios', '61', 'iphone', 'bad', 'iphone5', 'goo...
428    ['bad', 'thing', 'xiaomiredminote4', 'auto', '...
429    [' ', 'concern', 'service', 'provide', 'purcha...
430    ['buy', 'nokia61plus', 'oct2018', 'have', 'bad...
431    ['switch', 'oneplus_in', 'oneplus6', 'frustrat...
Name: bag_of_words, Length: 432, dtype: object

In [23]:
Test_X_Tfidf = Tfidf_vect.transform(df_test["bag_of_words"])

Brand Extractor


In [24]:
brand_list = ['Accer', 'Alcatel', 'AllView', 'Amazon', 'Amoi', 'Apple', 'Archos', 'Asus', 'AT&T', 'Benefon', 'BenQ', 'BenQ-Siemens', 'Bird', 'BlackBerry', 'BLU', 'Bosh', 'BQ', 'Casio', 'Cat', 'Celkon', 'Chea', 'CoolPad', 'Dell', 'Emporia', 'Energizer', 'Ericsson', 'Eten', 'Fujitsu Siemens', 'Garmin-Asus', 'Gigabyte', 'Gionee', 'Google', 'Haier', 'HP', 'HTC', 'Huawei', 'i-mate', 'i-mobile', 'Icemobile', 'Innostream', 'INQ', 'Intex', 'Jolla', 'Karbonn', 'Kyocera', 'Lava', 'LeEco', 'Lenovo', 'LG', 'Maxon', 'Maxwest', 'Meizu', 'Micromax', 'Microsoft', 'Mitac', 'Mitsubishi', 'Modu', 'Motorola', 'MWg', 'NEC', 'Neonode', 'NIU', 'Nokia', 'Nvidia', 'O2', 'OnePlus', 'Oppo', 'Orange', 'Palm', 'Panasonic', 'Pantech', 'Parla', 'Philips', 'Plum', 'Posh', 'Prestigio', 'QMobile', 'Qtek', 'Sagem', 'Samsung', 'Sendo', 'Sewon', 'Sharp', 'Siemens', 'Sonim', 'Sony', 'Sony Ericsson', 'Spice', 'T-Mobile', 'Tel.Me.', 'Telit', 'Thuraya', 'Toshiba', 'Unnecto', 'Vertu', 'Verykook', 'Vivo', 'VK Mobile', 'Vodafone', 'Wiko', 'WND', 'Xcute', 'Xiaomi', 'XOLO', 'Yezz', 'Yota', 'Yu', 'ZTE']


In [25]:
def extract_using_nlp(text):

    nlp_text = nlp(text)
    noun_chunks = nlp_text.noun_chunks
    # removing stop words and implementing word tokenization
    tokens = [token.text for token in nlp_text if not token.is_stop]
    # data = pd.read_csv(path) 
    
    # spec =  list(data['name'])

    spec = [item.lower() for item in brand_list]
    # print(spec)

    specset = []
    
    # check for one-grams (example: python)
    for token in tokens:
        if token.lower() in spec:
            specset.append(token)
    
    # check for bi-grams and tri-grams (example: machine learning)
    for token in noun_chunks:
        token = token.text.lower().strip()
        if token in spec:
            specset.append(token)
    
    return [i.capitalize() for i in set([i.lower() for i in specset])]
    
    
def find_organisation(text):
        
    
    doc = nlp(text)
    
    return [(X.text).capitalize() for X in doc.ents if X.label_ == "ORG"] 

In [26]:
# brand = []
# for text in df_tweet["cleaned_text"]:
#   x = extract_using_nlp(text , brand_list)
#   print(x)

df_test["brands"] = df_test["cleaned_text"].apply(extract_using_nlp)




In [ ]:
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)
i =0
for brand in df_test["brands"]:

  if brand != [] and predictions_SVM[i] == 0:
    predictions_SVM[i] = 1
  i= i+1

predictions_SVM

# predictions_SVM
# Use accuracy_score function to get the accuracy
# print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, test_Y)*100)

SENTIMENT AnalYSIS

In [ ]:
!pip install googletrans==4.0.0rc1

In [ ]:
!pip install flair

In [ ]:
!pip install langdetect
!pip install google-trans-new

In [ ]:
from langdetect import detect
import time
from google_trans_new import google_translator  


translator = google_translator()  
def pleaseTranslate(text):
  # try:
  #     if detect(text) != 'en':
  #       text = translator.translate(text,lang_tgt='en') 
  # except :
  #       pass
  text = translator.translate(text,lang_tgt='en')
  return text

In [ ]:
df_test["translated_text"] = df_test["cleaned_text"].apply(pleaseTranslate) 

In [ ]:
df_test[250:300]

In [ ]:
import flair

In [ ]:
flair_sentiment = flair.models.TextClassifier.load('en-sentiment')
def useFlair(text):
  s = flair.data.Sentence(text)
  flair_sentiment.predict(s)
  total_sentiment = s.labels
  return total_sentiment

In [ ]:
df_test["sentiment"] = df_test["translated_text"].apply(useFlair) 

In [ ]:
df_test

Headline Generator

In [ ]:
!pip install transformers

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelWithLMHead

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('t5-base')
model = AutoModelWithLMHead.from_pretrained('t5-base', return_dict=True)

In [ ]:
def summarize(text):
  inputs = tokenizer.encode("summarize: " + text,
                          return_tensors='pt',
                          max_length=200,
                          truncation=True)
  summary_ids = model.generate(inputs, max_length=20, min_length=20, length_penalty=5., num_beams=2)
  summary = tokenizer.decode(summary_ids[0])
  return summary


In [ ]:
# df_test["Summary"] = df_test["cleaned_text"].apply(summarize)
x =0 
for text in df_article["cleaned_text"]:
  print(summarize(text))
  x = x+ 1
  if x == 5:
    break

In [ ]:
df_test